# Notebook to edit data

In [1]:
import pygsheets
import pandas as pd
import numpy as np
import datetime
import util
c = pygsheets.authorize(outh_file='other.json')



# Protected Range

In [ ]:
gc = pygsheets.authorize(outh_file='/home/denis/Program/work/sheets.googleapis.com-python.json')
print(gc.get_range('1B1vEqp6uK7x2B8pV0dSH3S9DetFesaDP8n83Ewy53Yk', 'A1:D3' ))
gridrange = {
  "sheetId": 0,
  "startRowIndex": 0,
  "endRowIndex": 2,
  "startColumnIndex": 0,
  "endColumnIndex": 2,
}

editors = {
  "users": [
    'ffbskt@gmail.com',
  ],
  "groups": [
  ],
  "domainUsersCanEdit": True,
}
request = {"addProtectedRange": {
            "protectedRange": {
                "range": gridrange,
                "editors": editors
            },

        }}
#print(gc.sh_batch_update('1B1vEqp6uK7x2B8pV0dSH3S9DetFesaDP8n83Ewy53Yk', request, None, False))

In [10]:
gc = pygsheets.authorize(outh_file='/home/denis/Program/work/sheets.googleapis.com-python.json')
wks = util.read_sheet('1s5vtpVMmWZQCuUOdBVoNCCAVK_7U6ax3uZn0ynzgB28')
df = wks.get_as_df()

In [18]:
def open_protected_today(key):
    col = datetime.datetime.now().strftime("%m-%d")
    ind = df.columns.get_loc(col)
    
    #delete protected range
    request = {
              "deleteProtectedRange": {
                "protectedRangeId": 1,
              }
            }
    try:
        gc.sh_batch_update(key, request, None, False)
    except:
        pass
    #set new range
    gridrange = {
      "sheetId": 0,
      "startRowIndex": 0,
      "endRowIndex": 50,
      "startColumnIndex": 0,
      "endColumnIndex": ind,
    }
    editors = {
      "users": [
        'ffbskt@gmail.com',
      ],
      "groups": [
      ],
      "domainUsersCanEdit": True,
    }
    request = {"addProtectedRange": {
                "protectedRange": {
                "protectedRangeId": 1,
                "range": gridrange,
                "editors": editors
            },

        }}
    gc.sh_batch_update(key, request, None, False)
    ##open col of protection (can edit by link)
    
open_protected_today('1s5vtpVMmWZQCuUOdBVoNCCAVK_7U6ax3uZn0ynzgB28')

In [2]:
def first_empty(df, col):
    r = df.loc[[col]] != ''
    for ind, val in enumerate(r.values[0]):
        if not val:
            return ind
first_empty()

TypeError: first_empty() missing 2 required positional arguments: 'df' and 'col'

# Copy sheet and add to graph

In [ ]:
wks = util.read_sheet('1zx2VAIuiqFP9lZxwMUxeEyih7qpa2p5Su4mOw_ilC4E')#.get_as_df()
c.create(title='Quise', parent_id='1ncBfOFctKPVRKBYwrc58U5_UaD-d0D30')
wks_copy = util.read_sheet(c.spreadsheetId)
wks_copy.set_dataframe(wks.get_as_df(), start='A1')

# Edit data in sheet

In [ ]:
df = wks_copy.get_as_df()
df = util.join_suname_to_init(df, crange=[2,5], rrange=None, remove=True)

In [ ]:
df = df.rename(index=str, columns={"Фамилия": "ФИО", 
                              "Адрес электронной почты": "Почта",
                             "Учебная группа": "Группа",
                             "Секция по физкультуре": "Секция",
                             "Медецинская группа, как на фотографии": "справка"
                             })
df

In [ ]:
# Add node_name of main list (which course and faculty)
NODE_DICT = {'Физический1': "закрытый_список_2017"}#'закр список'}  #, ''}
def find_node_name(key, node_dict):
    if key in node_dict:
        return node_dict[key]
    else:
        return ''
df['node_name'] = [find_node_name(str(a) + str(b), NODE_DICT) 
                   for a, b in zip(df['Факультет'], df['Курс'])]


In [ ]:
df

In [ ]:
wks_copy.set_dataframe(df, start='A1')

In [ ]:
c.spreadsheetId # for sinc add 
#https://docs.google.com/spreadsheets/d/https://docs.google.com/spreadsheets/d/ + id

# Multiple sinc

We have form with multiple faculty and groups. We edit keys and values to main standart at the begining and add column "node dict" to contain name of link at graph sheet. Now we take form as df, sort by "node dict", open link and sync.

In [ ]:
def open_node_by_name(node_name, graph_df):
    row = util.get_row_by_keys(graph_df, key_column=["node_name"], key_value=[node_name])
    link = row["node"].values[0]
    return util.read_sheet(util.get_sheet_id_from_link(link))

In [ ]:
def get_ind(df, colname, ifval):
    return [str(i[1]) for i in zip(df[colname] == ifval, df.index) if i[0]]

In [ ]:
import util
graph_df = util.read_sheet('1MeAYhENafzQMoTKDG-VOLI308ih1wl7SjMLbOkwJU4M').get_as_df()
args = {"to_key_colname":[ "ФИО", "Группа"], "to_values_colname": ["Секция"]}
unique = list(df["node_name"].unique())
if '' in unique:
    unique.remove('')
while unique:
    main_name = unique.pop()
    main_wks = open_node_by_name(main_name, graph_df)
    from_wks = df[df["node_name"] == main_name]
    util.sync_by_colname(from_wks=from_wks, to_wks=main_wks, **args)
    df_sync_ind = get_ind(df, "node_name", main_name)


In [ ]:
df["mark"] = 0

In [ ]:
import json
from util import read_sheet, get_sheet_id_from_link, sync_by_colname

"""
Go by graph and sync all linked sheets by args.
"""

#graph = read_sheet('1MeAYhENafzQMoTKDG-VOLI308ih1wl7SjMLbOkwJU4M').get_as_df()
graph = read_sheet('1q8z_9QDwSia1IMo7qvDdH2cui0-D5My0xkClopMWcqw').get_as_df() # TEST


for i, line in graph.iterrows():
    if line['Pass']:
        continue

    from_wks = read_sheet(get_sheet_id_from_link(line['node']))
    links = line['link'].split(',')
    args = json.loads(line['args'])
    for link in links:
        to_wks = read_sheet(get_sheet_id_from_link(link))
        print(args['to_values_colname'], line['node'], link)
        sync_by_colname(from_wks=from_wks, to_wks=to_wks, **args)

# 2018 Optimize lessons


In [11]:
import pygsheets
import pandas as pd
import util
import random
import hashlib

In [3]:
import importlib

importlib.reload(util)
importlib.reload(PandasForm)
importlib.reload(SyncGraph)

NameError: name 'PandasForm' is not defined

In [44]:
c = pygsheets.authorize(outh_file='other.json')


#sh = c.open_by_key('1cWN-sMoaV_5NjWTcjryRx-6uxi29K9KaWt5vHa9vQog')#"1-xmNwRbxBwcZSHem67cVhf-ilG_bh_g_MRvR96Q74nE")#
#wks = sh.sheet1


In [5]:
def safe_transfer_to_pd_by_col(wks, col_list):
    """
        take col one by one and create pd dataframe (more stable then get_as_df)
        col_list = [1, 2, 6...]
    """ 
    df = pd.DataFrame()
    for col in col_list:
        
        full_col = wks.get_col(col, include_empty=False)
        #print(col, full_col, full_col[0], len(full_col), df.shape[0])
        diff_coll = df.shape[0] - len(full_col) + 1
        if diff_coll > 0: 
            for i in range(diff_coll):
                full_col.append('')
            
        df[full_col[0]] = full_col[1:]
    return df

In [6]:
def score_to_int(sc_val):
    return int(sc_val.split('/')[0])

In [27]:
wks = c.open_by_key(key='1cWN-sMoaV_5NjWTcjryRx-6uxi29K9KaWt5vHa9vQog').sheet1
df = safe_transfer_to_pd_by_col(wks, [1,2,3,4,5,6,7,8]) # 7 problem

In [ ]:
def filter_df_with_propertis(df, propertys_function):
    

In [28]:
# Rename columns from test
col_to_rename = {"Номер учебной группы (101, 201, 201э)":'group', 
                 "Email Address":'key', # emeil hash
                 "Score":'test_rool', # transform to int
                 'Фамилия и инициалы (Пример: Петров А. Б.)':'suname',
                 "Медицинская группа (на загруженной справке)":'med'
                }
df.rename(columns=col_to_rename, inplace=True)

In [12]:
#email -> hash
def mail_hesh(mail):
    return hashlib.md5(mail.encode('UTF-8')).hexdigest()


In [39]:

df['key'] = df['key'].apply(mail_hesh)
df['key'] = df['key'].apply(str) #int to long for google docs
df['cource'] = df['group'].apply(get_cource)
#df['key'] = df['key'].apply(lambda x: 'a'+str(x))

In [40]:
def get_cource(group_number):
    if group_number:
        return str(group_number[0])
    return ''
    #return group_number or group_number[0]


In [30]:
#test score to int 
df['test_rool'] = df['test_rool'].apply(score_to_int)

In [41]:
df

,Timestamp,key,test_rool,suname,group,Факультет,Фото справки медосмотра,Медицинская группа (на загруженной справке),cource
0,9/4/2018 23:20:14,2a734d098452ae0f283b0589db437d06,23,Зимнякова П.Е.,218,Физифак,,основная,2
1,9/8/2018 13:36:33,ff63cf8168b362978f3d0f242109bac5,21,Король А.К.,205,Физифак,,основная,2
2,8/31/2018 20:54:31,1c0c95e1a97a6635367991b902b44def,1,,,,,,
3,8/31/2018 21:09:25,eba5ac70cb76ada5d3809e6095d048fe,1,,,,,,
4,8/31/2018 21:57:02,e52fd7700fb6fe94e8a060d7cb279ebe,3,,,,,,
5,9/8/2018 18:33:19,89e35c3e4995d4020489ca62fd0c2c03,24,Давыдов М.М.,210,Физифак,,основная,2
6,9/1/2018 10:25:44,3165b49d0d3a92fb655a0252292fec70,3,,,,,,
7,9/5/2018 21:05:23,944f2eae5dca3b39848fb33adc8811b4,13,Андреев А.А.,212,Физифак,,основная,2
8,9/8/2018 13:19:48,530df3ed0a0bc8a8a6c1c7ae376ed92a,19,Лабунов А.С.,209,Физифак,,основная,2
9,9/1/2018 19:02:57,57a3dbe1bbd0a7ec399bbfb856da8699,2,,,,,,


In [42]:
#get list with same property (as faculty and test_rool > 10 )
#TODO for group filter 201m?
ndf = util.get_row_by_keys(df, key_value=['Химфак', '2'], key_column=['Факультет','cource'])
ndf = util.get_row_by_keys(df, ['10',], ['test_rool'], ratio=">")

In [ ]:
# add dates to dataframe
col_name=['key', 'suname', 'test_rool', 'group']
dates = util.get_dates(3,3,7)
df2 = pd.DataFrame(
            [[''] * len(dates)], 
            index=df.index, 
            columns=dates
        )
ndf = pd.concat([ndf[col_name], df2], axis=1)

In [ ]:
##get needed col
#col_name=['key', 'suname', 'test_rool', 'group']
df#[col_name]

In [ ]:
import PandasForm, SyncGraph


In [ ]:
def search_id_in_list(list_sheets, name):
    for sheet in list_sheets:
        if sheet['name'] == name:
            return sheet['id']
    return None

In [ ]:
# check add key or create new form
folder_id='1OnF9ZpSiO5IYB4Ns15PZgsTc14DVRr3z'
tittle="hf2"
fc = PandasForm.FormCreator(title=tittle, folder_id=folder_id)
if fc.check_shet_in_folder(pg=c, sheet_name=fc.title):
    # update with insert key
    sg = SyncGraph.Graph(None)
    wks = c.open_by_key(search_id_in_list(c.list_ssheets(folder_id), tittle))
    sg.add_key(df, wks.sheet1, key_col=['key']) #.sheet1
    args = {"to_key_colname": ["key",], 
            "to_values_colname": ['suname', 'test_rool', 'group'],            
           }
    util.sync_by_colname(from_wks=df[col_name], to_wks=wks.sheet1, **args)
else:
    #create new
    fc.df = df #[col_name]
    fc.create_sheet(pg=c)

In [51]:

q = 'kamironova2012@yandex.ru'

In [52]:

hashlib.md5(q.encode('UTF-8')).hexdigest()

'9f38155d1f444829a49df7995ff083c1'

In [48]:
#util.get_dates(5,3,7)

# Generate teams

In [ ]:
import datetime
todaydate = datetime.datetime.now().strftime("%m-%d")
tomorow = df.columns[df.columns.get_loc(todaydate) + 1]
today_col_ind = df.columns.get_loc(todaydate)

In [ ]:
folder_id='1OnF9ZpSiO5IYB4Ns15PZgsTc14DVRr3z'
tittle = 'ff2'
wks = c.open_by_key(search_id_in_list(c.list_ssheets(folder_id), tittle))

In [ ]:
players_count = sum([int(s or 0) for s in freshdf[todaydate]])

In [ ]:
if players_count % 3 == 0:
    teams_count = players_count / 3 - 1
else:
    teams_count = players_count // 3

In [ ]:
team_names = "abcdefghigklmnopq"
players_count / 3
team_list = []
for i in team_names[:teams_count]:
    team_list.extend(list(i * 3))
team_list.extend(list('R' * (players_count % 3 or 3)))

In [ ]:
import random
random.shuffle(team_list)

In [ ]:
i = 0
for ind, v in enumerate(freshdf[todaydate]):
    if v:
        freshdf[tomorow][ind] = team_list[i]
        i += 1

In [ ]:
freshdf = wks.sheet1.get_as_df()

In [ ]:
freshdf['today'] = freshdf[todaydate]

In [ ]:
util.get_row_by_keys(freshdf, key_column=['today',], key_value=[1,])

In [ ]:
freshdf

In [ ]:
wks.sheet1.update_col(index=today_col_ind + 2,row_offset=1,values=freshdf[tomorow].values.tolist())

In [ ]:
#col location
today_col_ind = df.columns.get_loc(todaydate)
df[df.columns[today_col_ind]]

In [ ]:
fc = PandasForm.FormCreator(title="ff2", folder_id='1OnF9ZpSiO5IYB4Ns15PZgsTc14DVRr3z')
fc.df = df[col_name]
fc.create_sheet(pg=c)

In [ ]:
c.list_ssheets(parent_id='1OnF9ZpSiO5IYB4Ns15PZgsTc14DVRr3z')

In [ ]:
sh = c.open('ff2')

In [ ]:
sh.sheet1.get_as_df()

In [ ]:
score_to_int(df["Score"][0]) > 19

In [ ]:
from GDrive import GDrive
g = GDrive(sheet_id='1cWN-sMoaV_5NjWTcjryRx-6uxi29K9KaWt5vHa9vQog',
                   graph='1at8ivfcSDq3W_LFdwzFqVuYfQ27c18Ka-F1Cpen844g'
                   )
#g.generate(category=['Секция', 'Факультет'], keys=['ФИО', 'Группа'],
#                   current_folder='19PZiIxtt9nvLDHst4EZp-FMWYQO4yrKa',
#                   graph_wks=util.read_sheet('1ZR38L8tFMdUTODVQ0cdz79jPKX4xpNn7R11S82u5Jyk'))

# Add Points for Relay

In [21]:
from TransferFunctions import BasketballResults
bres = BasketballResults('1GUCw3XNS0zWYgjv6dUACc5IoteSYft3DRW58vrDCIKA')

In [7]:
import importlib

importlib.reload(TransferFunctions)

<module 'TransferFunctions' from '/home/denis/Program/graph2/TransferFunctions.py'>

In [4]:
import TransferFunctions

In [8]:
bres = TransferFunctions.BasketballResults('1GUCw3XNS0zWYgjv6dUACc5IoteSYft3DRW58vrDCIKA')
df = bres.get_df()


In [9]:
df

,Timestamp,ref1,ref23,faculty,relay_1,relay_2,relay_3,relay_4,relay_5,relay_6,relay_7,marked
0,9/14/2018 16:11:41,ffbskt@gmail.com,msufizra@gmail.com,Физики2,bce,eba,,,,,,
1,9/14/2018 16:55:35,ffbskt@gmail.com,,Физики2,abc,,,,,,,


In [36]:
ref_point = 2
# df 
for row in df.iterrows():
    ans = []
    #print(row[1][11])
    if row[1][11] and int(row[1][11]) < 12:
        print(row[10])
        if int(row[1][11]) == 1: # add ref keys
            ans.extend([(mail_hesh(q), ref_point) for q in row[1][2].split()]) 
            #mark wks 2
        for i in range(min(4, row[1][11]), 11):
            print(row[1][i])
            #ans.extend(#teampoints)
                
    elif:
        ans.append(ref_point)
        ans.extend([(mail_hesh(q), ref_point) for q in row[1][2].split()])
        
        

In [13]:
df.columns[[1]]

Index(['ref1'], dtype='object')

In [21]:
a = ''
[(mail_hesh(q), 2) for q in a.split()]

[]

In [42]:
[1,2] + [2, 7]

[1, 2, 2, 7]

In [ ]:
def add_points_by_keys(keys, edited_df, val_col, points, key_col='key'): #  df[2].split() + df[[0]]
    for key in keys:
        #ad .. apply( + == * points)

In [ ]:
def add points_by_mails(mails, edited_df, val_col, points, key_col='key'):
    keys = [mail_hesh(mail) for mail in mails]
    return add_points_by_keys(keys, edited_df, val_col, points, key_col)

In [ ]:
def add_points_by_teams(team_places_str, team_key_dict): #f(place)
    for place, team in enumerate(team_places_str.split()):
        keys = team_key_dict[team]
        add_points_by_keys(keys, edited_df, val_col, 1 - place * 0.25, key_col)

In [ ]:
fac_to_sheet = {'Физики1':'addid',
                'Физики2':'',
                'Экономисты1':'',
                'Экономисты2':'',
                'Химики1':'addid',
                'Химики2':'1nYVmnNtnTewo8lxBklCTe1GRSzdxKgJYXgXdqiVXW0g'
               }

In [ ]:
def read_teams_result(relay_wks):
    df = relay_wks.get_as_df()
    
    


In [54]:
df['Timestamp'].apply(lambda x: '-'.join(x.split('/')[0:2]))

0    9-14
1    9-14
Name: Timestamp, dtype: object

In [58]:
datetime.datetime(df['Timestamp'])

TypeError: cannot convert the series to <class 'int'>

In [ ]:
self.df = self.safe_load(self.col_list)
        self.df.rename(columns=self.col_to_rename, inplace=True)

# Protected Range

In [54]:
c = pygsheets.authorize(outh_file='other.json')


In [55]:
wks = c.open('ch2').sheet1

In [70]:
wks.create_named_range("a", (1, 10), (5, 15))

AttributeError: 'DataRange' object has no attribute '_protected'

In [65]:
gridrange = {
  "sheetId": 0,
  "startRowIndex": 0,
  "endRowIndex": 2,
  "startColumnIndex": 0,
  "endColumnIndex": 2,
}
pr = wks.create_protected_range(gridrange)


In [ ]:
wks.create_named_range((1, 10), (5, 15))

In [71]:
wks.get_values((1, 10), (5, 15), returnas='range')

AttributeError: 'DataRange' object has no attribute '_protected'

In [ ]:
pr.update()